In [1]:
from shelve import open as open_shelve
from matplotlib import pyplot as plt
%matplotlib qt
from kitt_net import FeedForwardNet
from sklearn.metrics import confusion_matrix, accuracy_score
from numpy import newaxis, zeros

In [2]:
dataset = open_shelve('../examples/mnist/dataset_mnist_2K.ds')

In [1]:
how = {1: 'b.', -1: 'r.'}
for x, y in zip(dataset['x'], dataset['y']):
    plt.plot(x[0], x[1], how[y])
plt.grid()
plt.show()

NameError: name 'dataset' is not defined

In [3]:
net = FeedForwardNet(hidden=[20], tf_name='Sigmoid')

In [ ]:
n_epoch = 3
learning_rate = 0.3
net.fit(x=dataset['x'], y=dataset['y'], x_val=dataset['x_val'], y_val=dataset['y_val'],
       learning_rate=learning_rate, n_epoch=n_epoch, batch_size=16)


--------------------------------------------------------------------
-- Network initialized.
	% problem dimension: 784
	% number of classes: 10
	% class labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
	% net structure: [784, 20, 10]
	% net transfer function: Sigmoid

--------------------------------------------------------------------
-- Learning has started...
	% problem dimension: 784
	% number of training samples: 20000
	% number of validation samples: 10000
	% learning rate: 0.3
	% mini-batch size: 16
	% maximum number of epochs (t.c.): 3
	% maximum number of stable epochs (t.c.): inf
	% required accuracy (t.c.): inf
	% required error (t.c.): -inf


epoch   on training data      on validation data       epoch time          
-------------------------------------------------------------------


In [20]:
# Karnin sensitivities
s = [zeros(shape=w_i.shape) for w_i in net.w]
for ep_i in range(n_epoch):
    for l_i in range(len(s)):
        s[l_i] += net.dw_container[l_i][ep_i]**2 * (net.w[l_i]/(learning_rate*(net.w[l_i]-net.w_init[l_i])))

print s

[array([[ 4144.69669904,  8142.66407088,  -181.22122115,   192.60513786],
       [ 4355.77392173,  2038.69191119,  -123.66912155,    17.72412337],
       [  916.08618035,   272.07482858,   332.18451858,   692.9974425 ]]), array([[ 11531.50754774,  10746.17695456,   2411.70462884],
       [  9204.65668354,   6555.84540512,   2473.74145156]])]


In [6]:
net.prune(req_acc=0.88, n_epoch=5, measure='karnin')


--------------------------------------------------------------------
-- Pruning has started...
	% net initial structure: [784, 18, 10]
	% net initial number of synapses (w, b): (3573, 28)
	% min required accuracy (s.c.): 0.88
	% max required error (s.c.): 0.0
	% maximum number of re-training epochs (gu.c.): 5
	% number of stable iterations (gu.c.): 10


step    trying to cut     structure           left synapses (w/b)    retrained      next level     step time      
--------------------------------------------------------------------------------------------------------------
0       0                 [784, 18, 10]       (3573, 28)             None           75             None  


/home/kitt/mt_ws/py/kitt_lib/kitt_optimization.py:72: RuntimeWarning: divide by zero encountered in divide
  changes_[l_i] += self.vars['net_tmp'].dw_container[l_i][ep_i]**2*(self.vars['net_tmp'].w[l_i]/den)


ValueError: operands could not be broadcast together with shapes (20,784) (18,784) 

In [11]:
labels = [label for label in net.labels if label in dataset['y_val']]
y_pred = [net.predict(x)[0][0] for x, y in net.v_data]
print 'Acc:', accuracy_score(y_true=dataset['y_val'], y_pred=y_pred)
cm = confusion_matrix(y_true=dataset['y_val'], y_pred=y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, newaxis]
plt.imshow(cm, aspect='auto', interpolation='none', vmin=0, vmax=1)
plt.xticks(range(len(labels)), labels)
plt.yticks(range(len(labels)), labels)
plt.grid()
plt.colorbar()
plt.show()
print cm
for i in range(cm.shape[0]):
    print i, net.labels[i], sorted([(net.labels[j], round(cm[i, j], 2)) for j in range(cm.shape[1])], key=lambda x:x[1], reverse=True)[:5]


Acc: 0.98
[[ 1.    0.  ]
 [ 0.04  0.96]]
0 -1.0 [(-1.0, 1.0), (1.0, 0.0)]
1 1.0 [(1.0, 0.96), (-1.0, 0.04)]
